In [ ]:
# Imports


In [ ]:
# Class definition for S-Puzzle
class S_Puzzle:
    def __init__(self, initial_state):
        # Assume input initial_state is valid
        self.__size = len(initial_state)
        self.set_state(initial_state)
    
    def get_size(self):
        return self.__size
    
    def get_state(self):
        return self.__state
    
    def set_state(self, state):
        # Assume input state is valid, and same size as self.__size
        self.__state = state
        
    def get_current_state_children(self):
        children = []
        n = self.__size
        state = self.__state
        
        # Horizontal swaps
        for row in range(n):
            row_tuple = self.__state[row]
            for col in range(n-1):
                new_row = row_tuple[:col] + (row_tuple[col+1],) + (row_tuple[col],) + row_tuple[col+2:]
                new_tuple = state[:row] + (new_row,) + state[row+1:]
                children.append(new_tuple)
        
        # Vertical swaps
        for row in range(n-1):
            row1 = state[row]
            row2 = state[row+1]
            for col in range(n):
                new_row1 = row1[:col] + (row2[col],) + row1[col+1:]
                new_row2 = row2[:col] + (row1[col],) + row2[col+1:]
                new_tuple = state[:row] + (new_row1,) + (new_row2,) + state[row+2:]
                children.append(new_tuple)
        
        return children

In [ ]:
puzzle = S_Puzzle( ((6, 1, 2), (7, 8, 3), (5, 4, 9)) )
puzzle.get_current_state_children()